In [2]:
import os
import numpy as np
import pandas as pd
import yfinance as yf
from typing import List
from pandas_datareader import data as pdr
import gudhi as gd

# Variables globales

TICKERS = ["AAPL","MSFT","GOOGL","AMZN","NVDA","META","TSM","LLY","BRK-B","NVO"]
START = "2019-01-01"
END   = "2025-11-01"

OUT_DIR = "./data"
OUT_PARQUET = "market_panel.parquet"

AUTO_ADJUST = True   # yfinance: ajusta por splits/dividendos
BACK_ADJUST = False

INCLUDE_VIX = True
RV_WINDOWS = (5, 10, 21)   # días hábiles

# Mapeo estático simple de sector (puedes refinarlo con una fuente externa)
SECTOR_MAP = {
    "AAPL": "Tech", "MSFT": "Tech", "GOOGL": "Tech", "AMZN": "ConsDisc",
    "NVDA": "Tech", "META": "CommServ", "TSM": "Semis", "LLY": "Health",
    "BRK-B": "Financials", "NVO": "Health"
}


In [3]:
def asegurar_directorio_salida(path):
    if not os.path.exists(path):
        os.makedirs(path)


def download_adjusted_closes(
    tickers: List[str],
    start: str,
    end: str,
    auto_adjust: bool = True,
    threads: bool = True,
) -> pd.DataFrame:
    """
    Descarga precios 'Close' ajustados (dividendos/splits) desde Yahoo Finance.
    Devuelve un DataFrame con índice datetime y una columna por ticker.
    """
    if yf is None:
        raise ImportError("yfinance no está disponible. Instala con: pip install yfinance")

    if not tickers:
        raise ValueError("La lista de tickers no puede estar vacía.")

    data = yf.download(
        tickers,
        start=start,
        end=end,
        auto_adjust=auto_adjust,
        progress=False,
        group_by="ticker",
        threads=threads,
    )

    if isinstance(tickers, str) or len(tickers) == 1:
        t = tickers[0] if isinstance(tickers, list) else tickers
        closes = data["Close"].to_frame(name=t)
    else:
        closes = pd.DataFrame({t: data[t]["Close"] for t in tickers})
    closes = closes.sort_index().dropna(how="all")
    closes.index = pd.to_datetime(closes.index)

    # Opcional: asegurar zona horaria naive
    closes.index = closes.index.tz_localize(None) if getattr(closes.index, "tz", None) else closes.index
    return closes

def closes_wide_to_long_panel(closes_wide):
    """
    Convierte el DataFrame ancho (index: ds, cols: tickers con Close ajustado)
    a formato largo Nixtla-style con columnas: [unique_id, ds, Close].
    """
    if not isinstance(closes_wide.index, pd.DatetimeIndex):
        closes_wide.index = pd.to_datetime(closes_wide.index)

    
    # Mantén filas aunque haya NaNs (ya filtraremos luego)
    long = (closes_wide
            .stack(dropna=False)                # MultiIndex (ds, unique_id)
            .rename("Close")
            .reset_index()
            .rename(columns={"level_0":"ds", "level_1":"unique_id"}))
    # Tipos consistentes
    long["ds"] = pd.to_datetime(long["Date"])
    long["unique_id"] = long["unique_id"].astype(str)
    long["Close"] = pd.to_numeric(long["Close"], errors="coerce")

    # Orden canónico
    long = long.sort_values(["unique_id","ds"]).reset_index(drop=True)
    return long


# 2) AJUSTE: calcular_logret (quita print de debug)
def calcular_logret(df, price_col="Close"):
    out = df.copy()
    out = out.loc[:, ~out.columns.duplicated()].copy()

    if price_col not in out.columns:
        raise KeyError(f"No se encontró columna '{price_col}' en el DataFrame.")

    out = out.sort_values(["unique_id", "ds"]).copy()
    out[price_col] = pd.to_numeric(out[price_col], errors="coerce")

    prev = out.groupby("unique_id", sort=False)[price_col].shift(1)
    out["logret_raw"] = np.log(out[price_col] / prev)
    out.loc[~np.isfinite(out["logret_raw"]), "logret_raw"] = np.nan

    out["logret_scaled"] = np.nan
    for uid, idx in out.groupby("unique_id", sort=False).groups.items():
        vals = out.loc[idx, "logret_raw"].to_numpy()
        mask = np.isfinite(vals)
        if mask.sum() <= 1:
            out.loc[idx, "logret_scaled"] = vals
            continue
        x = vals[mask]
        med = np.median(x)
        q75 = np.percentile(x, 75)
        q25 = np.percentile(x, 25)
        iqr = q75 - q25
        if not np.isfinite(iqr) or iqr == 0:
            std = np.std(x)
            if not np.isfinite(std) or std == 0:
                std = 1.0
            scaled = (vals - med) / std
        else:
            scaled = (vals - med) / iqr
        out.loc[idx, "logret_scaled"] = scaled

    return out



def agregar_calendario(df):
    """
    futr_exog derivadas de la fecha.
    Arreglo:
      - Corregido 'dt' (antes estaba 't') y 'is_quarter_start' compatible.
    """
    out = df.copy()
    out["ds"] = pd.to_datetime(out["ds"])

    out["dow"]   = out["ds"].dt.weekday.astype("int16")
    out["dom"]   = out["ds"].dt.day.astype("int16")
    out["woy"]   = out["ds"].dt.isocalendar().week.astype("int16")
    out["month"] = out["ds"].dt.month.astype("int16")
    out["qtr"]   = out["ds"].dt.quarter.astype("int16")
    out["eom"]   = out["ds"].dt.is_month_end.astype("int8")
    out["eoq"]   = out["ds"].dt.is_quarter_end.astype("int8")
    out["eoy"]   = out["ds"].dt.is_year_end.astype("int8")
    return out



def agregar_hist_exog_mercado(df_panel, include_vix=True, rv_windows=(5,10,21)):
    """
    hist_exog:
      - vix (mismo para todas las series por fecha)
      - rv_*: volatilidad realizada por ticker (rolling std de logret_raw)
    """
    out = df_panel.copy().sort_values(["unique_id","ds"])

    # VIX
    if include_vix:
        vix = pdr.DataReader("VIXCLS", "fred", START, END)
        print(vix)
        vix = vix[["VIXCLS"]].reset_index().rename(columns={"DATE":"ds","VIXCLS":"vix"})
        vix["ds"] = pd.to_datetime(vix["ds"])
        out = out.merge(vix, on="ds", how="left")

    else:
        out["vix"] = np.nan

    # Realized volatility por unique_id
    for w in rv_windows:
        col = f"rv_{w}"
        # rolling std sobre logret_raw por serie
        out[col] = (
            out.groupby("unique_id", sort=False)["logret_raw"]
               .transform(lambda s: s.rolling(window=w, min_periods=max(2, w//2)).std())
               .astype(float)
        )
    return out


def agregar_stat_exog(df_panel, sector_map):
    """stat_exog: sector constante por unique_id."""
    out = df_panel.copy()
    out["sector"] = out["unique_id"].map(sector_map).fillna("Unknown")
    return out


def finalizar_panel(df):
    """
    Ordena y selecciona columnas finales:
      - Core: unique_id, ds, y
      - futr_exog: calendario
      - hist_exog: vix, rv_*
      - stat_exog: sector
      - Auditoría: Close, Volume, logret_raw
    """
    df = df.rename(columns={"logret_scaled": "y"})
    core = ["unique_id","ds","y"]

    futr = ["dow","dom","woy","month","qtr","eom","eoq","eoy"]
    futr = [c if c in df.columns else None for c in futr]
    futr = [c for c in futr if c is not None]

    hist = ["vix"] + [c for c in df.columns if c.startswith("rv_")]
    # stat = ["sector"]
    audit = [c for c in ["Close","Volume","logret_raw"] if c in df.columns]

    ordered = core + futr + hist + audit
    ordered = [c for c in ordered if c in df.columns]

    out = df[ordered].copy()
    out = out.sort_values(["unique_id","ds"]).reset_index(drop=True)

    # Tipos
    out["unique_id"] = out["unique_id"].astype(str)
    out["ds"] = pd.to_datetime(out["ds"])
    out["y"] = out["y"].astype(float)
    return out


def construir_panel():
    # 1) Descarga wide (1 col por ticker)
    closes_wide = download_adjusted_closes(
        TICKERS,
        START,
        END,
        auto_adjust=AUTO_ADJUST,  # correcto
        threads=True              # NO pasar BACK_ADJUST aquí
    )

    # 2) wide -> long (unique_id, ds, Close)
    precios = closes_wide_to_long_panel(closes_wide)

    # 3) Log-returns + escalado robusto
    ret = calcular_logret(precios, price_col="Close")

    # 4) Exógenas de calendario (futr_exog)
    ret = agregar_calendario(ret)

    # 5) Exógenas históricas (VIX + realized vol)
    ret = agregar_hist_exog_mercado(ret, include_vix=INCLUDE_VIX, rv_windows=RV_WINDOWS)

    # 6) Exógenas estáticas por serie (sector)
    ret = agregar_stat_exog(ret, SECTOR_MAP)

    # 7) Finalizar columnas y limpieza básica
    panel = finalizar_panel(ret)
    panel = panel.dropna(subset=["y"]).reset_index(drop=True)
    return panel


def guardar_panel(df):
    asegurar_directorio_salida(OUT_DIR)
    ruta = os.path.join(OUT_DIR, OUT_PARQUET)
    df.to_parquet(ruta, index=False)
    return ruta


def imprimir_esquema_y_listas(df):
    # Esquema
    esquema = pd.DataFrame({"column": df.columns, "dtype": [str(df[c].dtype) for c in df.columns]})
    print("Schema (columna, dtype):")
    print(esquema.to_string(index=False))

    # Listas sugeridas para NBEATSx
    futr_exog = [c for c in ["dow","dom","woy","month","qtr","eom","eoq","eoy","is_month_start","is_quarter_start","is_year_start"] if c in df.columns]
    hist_exog = ["vix"] + [c for c in df.columns if c.startswith("rv_")]
    stat_exog = ["sector"]

    # print("\nSugerencia de exógenas para NeuralForecast.NBEATSx:")
    # print("  futr_exog =", futr_exog)
    # print("  hist_exog =", hist_exog)
    # print("  stat_exog =", stat_exog)


In [4]:
panel = construir_panel()
ruta = guardar_panel(panel)
print("Panel estilo Nixtla guardado en:", ruta)
# imprimir_esquema_y_listas(panel)


KeyboardInterrupt: 

In [ ]:
prueba = pd.read_parquet(r"C:\Users\HP\Downloads\Tareas y actividades Tec\Experimento_2_NIXTLA\data\market_panel.parquet")
prueba.columns

Index(['unique_id', 'ds', 'y', 'dow', 'dom', 'woy', 'month', 'qtr', 'eom',
       'eoq', 'eoy', 'vix', 'rv_5', 'rv_10', 'rv_21', 'Close', 'logret_raw'],
      dtype='object')

# Agregar TDA

En esta sección buscaremos hacer las pruebas de la 

In [ ]:
def takens_embedding_1d(series, m, tau):
    """
    Construye la incrustación de Takens para una serie 1D.

    Parámetros
    ----------
    series : array-like (1D)
        Serie temporal (ej. logret_scaled) como np.ndarray.
    m : int
        Dimensión de incrustación.
    tau : int
        Retardo (lag) entre coordenadas.

    Returns
    -------
    emb : np.ndarray of shape (n_points, m)
        Nube de puntos embebida. Si la serie es demasiado corta,
        devuelve un array vacío de shape (0, m).
    """
    series = np.asarray(series, dtype=float)
    n = series.shape[0]
    delay_span = (m - 1) * tau

    if n <= delay_span:
        return np.empty((0, m))

    n_points = n - delay_span
    emb = np.empty((n_points, m), dtype=float)

    for i in range(n_points):
        for j in range(m):
            emb[i, j] = series[i + j * tau]

    return emb


# =========================
# 2. Diagrama de persistencia H1 con Gudhi
# =========================

# def compute_h1_diagram_rips(points, maxdim=1):
#     """
#     Calcula el diagrama de persistencia en homología 1 (H1) usando
#     un complejo de Rips de Gudhi.

#     Parámetros
#     ----------
#     points : np.ndarray, shape (n_points, n_dims)
#         Nube de puntos en R^d.
#     maxdim : int
#         Dimensión máxima del complejo de Rips (>= 1).

#     Returns
#     -------
#     diag_h1 : np.ndarray, shape (n_features, 2)
#         Arreglo con [birth, death] de cada clase en H1.
#         Si no hay puntos o no hay ciclos, devuelve shape (0, 2).
#     """
#     points = np.asarray(points, dtype=float)

#     if points.ndim != 2 or points.shape[0] < 2:
#         # Con menos de 2 puntos no hay nada interesante que calcular
#         return np.empty((0, 2))

#     rips_complex = gd.RipsComplex(points=points)
#     simplex_tree = rips_complex.create_simplex_tree(max_dimension=maxdim)

#     # homology_coeff_field=2 es el default; min_persistence se podría tunear
#     simplex_tree.compute_persistence()

#     diag_h1 = simplex_tree.persistence_intervals_in_dimension(1)

#     if len(diag_h1) == 0:
#         return np.empty((0, 2))

#     diag_h1 = np.asarray(diag_h1, dtype=float)

#     # Filtrar muertes infinitas, si las hubiera
#     finite_mask = np.isfinite(diag_h1[:, 1])
#     diag_h1 = diag_h1[finite_mask]

#     if diag_h1.shape[0] == 0:
#         return np.empty((0, 2))

#     return diag_h1

def compute_h1_diagram_rips(points, maxdim=2):
    """
    Calcula el diagrama de persistencia en homología 1 (H1) usando
    un complejo de Rips de Gudhi.
    """
    points = np.asarray(points, dtype=float)

    if points.ndim != 2 or points.shape[0] < 2:
        return np.empty((0, 2))

    # AHORA max_dimension=2 para que se construyan triángulos
    rips_complex = gd.RipsComplex(points=points)
    simplex_tree = rips_complex.create_simplex_tree(max_dimension=maxdim)

    simplex_tree.compute_persistence()
    diag_h1 = simplex_tree.persistence_intervals_in_dimension(1)

    if len(diag_h1) == 0:
        return np.empty((0, 2))

    diag_h1 = np.asarray(diag_h1, dtype=float)

    # El resto lo dejamos igual por esta prueba
    finite_mask = np.isfinite(diag_h1[:, 1])
    diag_h1 = diag_h1[finite_mask]

    if diag_h1.shape[0] == 0:
        return np.empty((0, 2))

    return diag_h1


# =========================
# 3. Estadísticos TDA sobre un diagrama
# =========================

def persistent_entropy(diagram, eps=1e-12):
    """
    Entropía persistente de un diagrama de persistencia.

    Definición:
    - Sea d_i = death_i - birth_i > 0
    - L = sum_i d_i
    - p_i = d_i / L
    - H = - sum_i p_i * log(p_i)

    Si el diagrama está vacío o L <= 0, devuelve 0.0.

    Parámetros
    ----------
    diagram : np.ndarray, shape (n_features, 2)
        Diagrama de persistencia [birth, death].
    eps : float
        Pequeño valor para evitar log(0).

    Returns
    -------
    h : float
        Entropía persistente (log natural).
    """
    if diagram.size == 0:
        return 0.0

    lifetimes = diagram[:, 1] - diagram[:, 0]
    lifetimes = lifetimes[lifetimes > 0.0]

    if lifetimes.size == 0:
        return 0.0

    L = np.sum(lifetimes)
    if L <= 0.0:
        return 0.0

    p = lifetimes / L
    p = p[p > 0.0]

    if p.size == 0:
        return 0.0

    h = -np.sum(p * np.log(p + eps))
    return float(h)


def diagram_amplitude(diagram, p=2.0):
    """
    Amplitud de un diagrama de persistencia.

    Implementación estándar:
    - Distancia de cada punto a la diagonal en norma L∞ es d_i / 2,
      donde d_i = death_i - birth_i.
    - Definimos la amplitud como:
          A_p = ( sum_i (d_i / 2)^p )^(1/p)

    Si el diagrama está vacío, devuelve 0.0.

    Parámetros
    ----------
    diagram : np.ndarray, shape (n_features, 2)
        Diagrama de persistencia [birth, death].
    p : float
        Parámetro de la norma (habitualmente p=2).

    Returns
    -------
    A_p : float
        Amplitud del diagrama.
    """
    if diagram.size == 0:
        return 0.0

    lifetimes = diagram[:, 1] - diagram[:, 0]
    lifetimes = lifetimes[lifetimes > 0.0]

    if lifetimes.size == 0:
        return 0.0

    distances = lifetimes / 2.0  # distancia a la diagonal
    if p <= 0:
        # Por seguridad, tratamos p<=0 como caso límite: usar máximo
        return float(np.max(distances))

    A_p = np.sum(distances ** p) ** (1.0 / p)
    return float(A_p)


def diagram_num_points(diagram):
    """
    Número de puntos en el diagrama (cardinalidad).

    Parámetros
    ----------
    diagram : np.ndarray, shape (n_features, 2)

    Returns
    -------
    n_points : int
    """
    return int(diagram.shape[0])


# =========================
# 4. Features TDA para UNA ventana
# =========================

def tda_features_for_window(window_values, m=4, tau=4, maxdim=1):
    """
    Calcula entropía persistente, amplitud y número de puntos
    sobre H1 de una ventana temporal de la serie (logret_scaled).

    Parámetros
    ----------
    window_values : array-like (1D)
        Segmento de la serie temporal.
    m : int
        Dimensión de Takens.
    tau : int
        Retardo de Takens.
    maxdim : int
        Dimensión máxima del complejo de Rips (>=1).

    Returns
    -------
    entropy : float
    amplitude : float
    n_points : int
        Si no se pueden calcular (ej. ventana muy corta o NaNs),
        devuelve (np.nan, np.nan, np.nan).
    """
    window_values = np.asarray(window_values, dtype=float)

    # Si hay NaNs, preferimos no calcular nada
    if np.isnan(window_values).any():
        return np.nan, np.nan, np.nan

    emb = takens_embedding_1d(window_values, m=m, tau=tau)

    if emb.shape[0] == 0:
        return np.nan, np.nan, np.nan

    diag_h1 = compute_h1_diagram_rips(emb, maxdim=maxdim)

    # Si el diagrama es vacío, devolvemos 0.0 para las métricas (sin NaNs)
    if diag_h1.shape[0] == 0:
        return 0.0, 0.0, 0.0

    entropy = persistent_entropy(diag_h1)
    amplitude = diagram_amplitude(diag_h1, p=2.0)
    n_points = diagram_num_points(diag_h1)

    return entropy, amplitude, float(n_points)


# =========================
# 5. Aplicar TDA al panel Nixtla (ventana 21 días, H1)
# =========================

def add_tda_helpers_to_panel(
    df,
    value_col="logret_scaled",
    id_col="unique_id",
    time_col="ds",
    window_size=21,
    m=4,
    tau=4
):
    """
    Agrega 3 columnas TDA al panel Nixtla, calculadas a partir de
    logret_scaled usando ventana deslizante de 21 días y homología H1.

    Nuevas columnas:
    - 'tda_entropy_h1_w21'
    - 'tda_amplitude_h1_w21'
    - 'tda_n_points_h1_w21'

    Parámetros
    ----------
    df : pd.DataFrame
        DataFrame con columnas tipo Nixtla [unique_id, ds, y, ..., logret_scaled].
    value_col : str
        Nombre de la columna sobre la que se calcula Takens (ej. 'logret_scaled').
    id_col : str
        Columna de identificador de serie (ej. 'unique_id').
    time_col : str
        Columna temporal (ej. 'ds').
    window_size : int
        Tamaño de la ventana deslizante (21).
    m : int
        Dimensión de Takens.
    tau : int
        Retardo de Takens.

    Returns
    -------
    df_out : pd.DataFrame
        Nuevo DataFrame con las columnas TDA añadidas.
    """
    # Copia para no modificar el original in-place
    df = df.copy()

    # Ordenamos por panel y tiempo para asegurar causalidad
    df = df.sort_values([id_col, time_col]).reset_index(drop=True)

    # Inicializamos columnas de salida
    df["tda_entropy_h1_w21"] = np.nan
    df["tda_amplitude_h1_w21"] = np.nan
    df["tda_n_points_h1_w21"] = np.nan

    # Procesamos serie por serie (por ticker)
    for uid, group_idx in df.groupby(id_col).groups.items():
        idx_array = np.asarray(list(group_idx))
        series_values = df.loc[idx_array, value_col].to_numpy(dtype=float)

        n = series_values.shape[0]

        ent_values = np.full(n, np.nan, dtype=float)
        amp_values = np.full(n, np.nan, dtype=float)
        npts_values = np.full(n, np.nan, dtype=float)

        for i in range(window_size - 1, n):
            window = series_values[i - window_size + 1 : i + 1]

            e, a, npts = tda_features_for_window(
                window,
                m=m,
                tau=tau,
                maxdim=2
            )

            ent_values[i] = e
            amp_values[i] = a
            npts_values[i] = npts

        # Asignamos de vuelta las columnas a las filas correspondientes
        df.loc[idx_array, "tda_entropy_h1_w21"] = ent_values
        df.loc[idx_array, "tda_amplitude_h1_w21"] = amp_values
        df.loc[idx_array, "tda_n_points_h1_w21"] = npts_values

    return df

In [ ]:
df = pd.read_parquet(r"C:\Users\HP\Downloads\Tareas y actividades Tec\Experimento_2_NIXTLA\data\market_panel.parquet")

# (Si logret_scaled ya existe, no necesitas esto; es solo un ejemplo)
# df["logret_scaled"] = ...

# Agregar helpers TDA
df_tda = add_tda_helpers_to_panel(
    df,
    value_col="y",
    id_col="unique_id",
    time_col="ds",
    window_size=21,
    m=4,
    tau=4
)

# Luego puedes volver a guardar:
df_tda.to_parquet(r"data\panel_tda.parquet")


NameError: name 'pd' is not defined